In [ ]:

import pandas as pd 
from collections import defaultdict
from itertools import combinations


#import csv
trump_insults = pd.read_csv("../input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv")

#create a list withing a list, where the inner-list is a tweet with individual words as items and the outer-list has each tweet as an item
master_list = []
insult_list = []
for tweet in trump_insults.tweet:
    insult_list.append(tweet)
for tweet in insult_list:
    word_list = list(tweet.split(" "))
    master_list.append(word_list)

#show how first five tweets in the master_list are structured
print(master_list[:5])


We need to create a list of common words that we want to exclude from our pairs. To create this list, I ran the program with a high threshold and a min count, then I added words I felt had no significance (a, the, or, and) added them to this list to be excluded when I excecute the update_pair_counts function.

In [ ]:
unsig_words =['in','and', '-', 'are', 'all', 'the', 'they', 'is', 'a', 'of', 'his', 'to', 'The', 'just', 'who', 'but', 'by', 'has', 'in', 'not',
             'their','will', 'that', '&', 'was','been', 'have', 'than', 'would', 'should', 'be','said', 'If', 'would', 'him', 'for', 'did', 'I', 'my', 'he',
              'He', 'on'] 

Define functions to make dictionaries of {word_pairs : count_of_word_pairs} and {word : occurence of word in item} for each tweet

In [ ]:

#write function to count n of times a specific set of words are together
def update_pair_counts (pair_counts, itemset):
    assert type (pair_counts) is defaultdict

    for a,b in combinations(itemset, 2):
        pair_counts[a,b] +=1
        pair_counts[b,a] +=1
        
#write a function to count n of times a word is used in all of the tweets
def update_item_counts(item_counts, itemset):
    assert type(item_counts) is defaultdict
    
    for a in itemset:
        item_counts[a] += 1
        
#write a function to filter rules by confidence
def filter_rules_by_confidence (pair_counts, item_counts, threshold):
    conf_rules = {} 
    for (a,b) in pair_counts:
        assert a in item_counts
        if a in unsig_words or b in unsig_words:
            continue
        conf_ab = pair_counts[(a, b)]/ item_counts[a]
        if conf_ab >= threshold:
            conf_rules[(a,b)] = conf_ab
    return conf_rules

Apply functions to master_list to generate confident word pairing rules

In [ ]:
#set confidence filter
conf_threshold = 0.50

#define default dictionaries for the functions below
pair_counts = defaultdict(int)
item_counts = defaultdict(int)

for itemsets in master_list:
    update_pair_counts(pair_counts, itemsets)
    update_item_counts(item_counts, itemsets)
tweet_word_rules = filter_rules_by_confidence(pair_counts, item_counts, conf_threshold)

#if we don't also filter by a minimum count of the rule occuring, then we'll get word-pairs that are always found together but may only be seen once or twice
high_freq_list = []
min_count = 150

for rule in tweet_word_rules.items():
    if item_counts[rule[0][0]] >= min_count:
        high_freq_list.append(rule)

    

high_freq_rules = dict(high_freq_list)
print('The function created',len(high_freq_rules),'rules with a confidence of', conf_threshold,'or higher and an occurrence requirement of', str(min_count)+'.')

Pretty print the code with the functions below.

In [ ]:
def gen_rule_str(a, b, val=None, val_fmt='{:.2f}', sep=" = "):
    text = "{} => {}".format(a, b)
    if val:
        text = "confidence(" + text + ")"
        text += sep + val_fmt.format(val)
    return text

def print_rules(rules):
    if type(rules) is dict or type(rules) is defaultdict:
        from operator import itemgetter
        ordered_rules = sorted(rules.items(), key=itemgetter(1), reverse=True)
    else: # Assume rules is iterable
        ordered_rules = [((a, b), None) for a, b in rules]
    for (a, b), conf_ab in ordered_rules:
        print(gen_rule_str(a, b, conf_ab))

        
print_rules(high_freq_rules)

The variables to play with in this are the insiginifcant words list, the minimum pair count, and the confidence threshold. The program generally returns two proper nouns mentioned in the same tweet, as you can see. Using the insignificant list can definitely skew the results. If you want more colorful pairings, just turn the minimum count down to 10 or so.